In [4]:
import pandas as pd
import numpy as np

import os 

import matplotlib.pyplot as plt
from PIL import Image 
from glob import glob

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Importing keras libraries and packages

from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import MaxPooling2D

In [5]:
train_path = '../input/fruits/fruits-360/Training/'
test_path = '../input/fruits/fruits-360/Test/'

In [7]:
# Model configuration
batch_size = 25
img_width, img_height, img_num_channels = 100, 100, 3
loss_function = sparse_categorical_crossentropy
no_epochs = 25
optimizer = Adam()
validation_split = 0.2
verbosity = 1

# Determine shape of the data
input_shape = (img_width, img_height, img_num_channels)

# Create a generator
datagen = ImageDataGenerator(
  rescale=1./255
)
train_datagen = datagen.flow_from_directory(
        train_path,
#         save_to_dir='./',
#         save_format='jpeg',
        batch_size=100,
        target_size=(100, 100),
        class_mode='sparse')
test_datagen = datagen.flow_from_directory(

        test_path,
#         save_to_dir='./',
#         save_format='jpeg',
        batch_size=100,
        target_size=(100, 100),
        class_mode='sparse')

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [8]:
# Create the model
model = Sequential()
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(.2))

model.add(Conv2D(32, kernel_size=(5, 5), activation='relu'))
model.add(Flatten())

model.add(Dense(16, activation='relu'))
model.add(Dense(131, activation='softmax'))

In [9]:
# Display a model summary
model.summary()

# Compile the model
model.compile(loss=loss_function,
              optimizer=optimizer,
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 16)        1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 48, 48, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 32)        12832     
_________________________________________________________________
flatten (Flatten)            (None, 61952)             0         
_________________________________________________________________
dense (Dense)                (None, 16)                991248    
_________________________________________________________________
dense_1 (Dense)              (None, 131)               2

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('weights.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# Start training
model.fit(
        train_datagen,
        steps_per_epoch=60000//200,
        validation_data=test_datagen,
        validation_steps=20000//100,
        callbacks=[checkpoint],
        epochs=10)

Epoch 1/10
300/300 [==============================] - 373s 1s/step - loss: 4.4759 - accuracy: 0.0386 - val_loss: 3.3328 - val_accuracy: 0.1580

Epoch 00001: val_accuracy improved from -inf to 0.15800, saving model to weights.hdf5
Epoch 2/10
280/300 [===========================>..] - ETA: 13s - loss: 2.9272 - accuracy: 0.1978

In [ ]:
model2=Sequential()
model2.add(Conv2D(32, (3,3), input_shape=input_shape,activation='swish'))
model2.add(MaxPooling2D())
model2.add(Dropout(.1))

model2.add(Conv2D(64, (3,3),activation='swish'))
model2.add(MaxPooling2D())
model2.add(Dropout(.1))

model2.add(Conv2D(128, (3,3),activation='swish'))
model2.add(MaxPooling2D())
model2.add(Dropout(.1))

model2.add(Conv2D(256,(3,3),activation='swish'))
model2.add(MaxPooling2D())
model2.add(Dropout(.1))


model2.add(Flatten())
model2.add(Dense(512,activation='swish'))
model2.add(Dropout(0.2))
model2.add(Dense(131,activation='softmax'))
# Display a model summary
model2.summary()

# Compile the model
model2.compile(loss=loss_function,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])
checkpoint = tf.keras.callbacks.ModelCheckpoint('weight2s.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

model2.fit(
        train_datagen,
        steps_per_epoch=60000//200,
        validation_data=test_datagen,
        validation_steps=20000//100,
        callbacks=[checkpoint],
        epochs=10)